# Nurgle

> "Buboes, phlegm, blood and guts! Boils, bogeys, rot and pus! Blisters, fevers, weeping sores! From your wounds the fester pours."
> 
> -- <cite>The Chant of Nurgle</cite>

The following analyses require `jupyterthemes`, `numpy`, `pandas`, and `plotly`. Install them with `pip`.

In [ ]:
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('grade3')

In [ ]:
import numpy
import pandas

In [ ]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## Metabolism

### Concentration of Compounds

Read timecourse data first.

In [ ]:
filename = "../build/timecourse.csv"
pddata = pandas.read_csv(filename)
pddata = pddata.rename(columns={pddata.columns[0]: "Time"})

Plot concentrations of compounds.

In [ ]:
def plot_scatter(df, filename, indices=None, nsteps=10):
    if indices is None:
        (m, _) = df.shape
        indices = range(0, m, m // nsteps)
    (ymin, ymax) = (min(df.iloc[indices , 1: ].values.flatten()), max(df.iloc[indices , 1: ].values.flatten()))
    (ymin, ymax) = ymin - (ymax - ymin) / 15, ymax + (ymax - ymin) / 15

    scatters = [
        dict(
            y=df.iloc[idx, 1: ],
            text=df.columns[1: ],
            mode='markers',
            marker=dict(
                size='12', color=df.iloc[idx, 1: ], colorscale='Viridis', showscale=True,
                cmin=ymin, cmax=ymax,
                line=dict(width=1)
                ),
            visible=False
            )
        for idx in indices]
    scatters[0]['visible'] = True

    steps = []
    for i, idx in enumerate(indices):
        step = dict(
            method='restyle',
            label='{}'.format(df.iloc[idx, 0]),
            args=['visible', [False] * len(scatters)],
            )
        step['args'][1][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [
        dict(
            active=0,
            currentvalue=dict(prefix="Time="),
            pad=dict(t=50),
            steps=steps
            )
        ]

    layout = dict(
        hovermode= 'closest',
        xaxis= dict(title='Compound'),
        yaxis=dict(title='Concentration', range=(ymin, ymax)),
        showlegend= False,
        sliders=sliders,
        height=600
        )

    fig = dict(data=scatters, layout=layout)
    iplot(fig, filename=filename)

In [ ]:
plot_scatter(pddata, "compound", nsteps=15)